Import Dependancies

In [3]:
# Dependencies
from config import (gkey)
#import gmaps
import numpy as np
import pandas as pd
import requests
import time
from scipy.stats import linregress
from matplotlib import pyplot as plt


#import cleaned county data
county_data = pd.read_csv("../Matthew/Data_csv/state_county.csv")

,Unnamed: 0,State,County,"Male life expectancy, 1985 (years)","Female life expectancy, 1985 (years)","Male life expectancy, 1990 (years)","Female life expectancy, 1990 (years)","Male life expectancy, 1995 (years)","Female life expectancy, 1995 (years)","Male life expectancy, 2000 (years)","Female life expectancy, 2000 (years)","Male life expectancy, 2005 (years)","Female life expectancy, 2005 (years)","Male life expectancy, 2010 (years)","Female life expectancy, 2010 (years)","Difference in male life expectancy, 1985-2010 (years)","Difference in female life expectancy, 1985-2010 (years)"
0,1,Alabama,NaN,69.1,76.9,69.4,77.1,69.9,77.2,71.0,77.2,71.5,77.3,72.4,77.8,3.3,0.9
1,2,Alabama,Autauga,68.1,77.0,68.5,77.0,69.3,77.3,71.7,77.6,72.5,77.4,73.3,78.8,5.2,1.8
2,3,Alabama,Baldwin,71.1,78.8,71.9,79.2,72.4,79.3,73.9,79.2,73.8,79.7,75.0,80.3,3.8,1.5
3,4,Alabama,Barbour,66.8,76.0,66.7,76.7,68.4,76.9,70.0,76.2,70.5,77.0,72.2,77.2,5.3,1.1
4,5,Alabama,Bibb,67.3,76.6,68.4,76.2,69.0,75.9,70.3,76.5,70.5,76.8,71.5,76.8,4.2,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3188,3189,Wyoming,Sweetwater,72.9,78.9,73.0,79.2,72.7,78.7,73.8,78.9,74.5,78.2,75.5,81.3,2.6,2.4
3189,3190,Wyoming,Teton,73.0,77.5,73.9,78.5,75.7,80.1,78.1,80.9,79.7,83.5,80.9,83.3,7.9,5.8
3190,3191,Wyoming,Uinta,73.5,79.5,73.6,78.8,74.0,79.1,74.5,79.3,75.1,79.5,74.8,79.9,1.3,0.4
3191,3192,Wyoming,Washakie,73.1,79.9,73.2,79.5,73.9,79.5,74.8,79.3,75.2,80.1,75.9,79.3,2.8,-0.7
